In [1]:

import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects

/home/medcoast/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [3]:
vos_files = pd.Series(
    sorted(glob("shoreline_data_run6/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
3064    shoreline_data_run6/usa_HI_0051/time_series_ti...
3065    shoreline_data_run6/usa_HI_0052/time_series_ti...
3066    shoreline_data_run6/usa_HI_0053/time_series_ti...
3067    shoreline_data_run6/usa_HI_0054/time_series_ti...
3068    shoreline_data_run6/usa_HI_0057/time_series_ti...
Length: 2509, dtype: object

In [4]:
my_nz_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_nz_files

0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 562, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
2540    data/sar2541/transect_time_series.csv
Length: 2541, dtype: object

In [6]:
files = pd.concat([vos_files, my_nz_files, sar_files])
files

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
2540                data/sar2541/transect_time_series.csv
Length: 5612, dtype: object

In [7]:
"""f = sar_files.iloc[0]
despiked_filename = f.replace(".csv", "_despiked.csv")
df = pd.read_csv(despiked_filename)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
import matplotlib.pyplot as plt
for transect_id in df.columns:
  sample = df[transect_id]
  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")
  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")
  ax.legend()
  plt.show()"""

'f = sar_files.iloc[0]\ndespiked_filename = f.replace(".csv", "_despiked.csv")\ndf = pd.read_csv(despiked_filename)\ndf.dates = pd.to_datetime(df.dates)\ndf.set_index("dates", inplace=True)\nimport matplotlib.pyplot as plt\nfor transect_id in df.columns:\n  sample = df[transect_id]\n  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")\n  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")\n  ax.legend()\n  plt.show()'

In [8]:
#trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
#len(trends)
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f:
        despiked_filename = f.replace(".csv", "_despiked.csv")
        try:
            raise
            df = pd.read_csv(despiked_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_despiked.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


#trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
#len(trends)
trends=get_trends(sar_files.iloc[-1]).set_index("transect_id")

In [9]:
trends

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
sar2541-0000,-0.315984,199.516726,1371,1305,0.049214,14.115068,283.782370,16.845841
sar2541-0001,0.031699,165.718466,1371,1328,0.001673,6.446574,87.725117,9.366169
sar2541-0002,0.073058,161.154798,1371,1329,0.007582,7.284243,102.875692,10.142766
sar2541-0003,-0.207761,182.216593,1371,1276,0.018165,14.834138,345.963697,18.600099
sar2541-0004,-1.407345,416.075469,1371,1275,0.491459,14.094369,309.068806,17.580353


In [10]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
sar2541-0001,0.031699,165.718466,1371,1328,0.001673,6.446574,87.725117,9.366169
sar2541-0002,0.073058,161.154798,1371,1329,0.007582,7.284243,102.875692,10.142766
sar2541-0003,-0.207761,182.216593,1371,1276,0.018165,14.834138,345.963697,18.600099
sar2541-0000,-0.315984,199.516726,1371,1305,0.049214,14.115068,283.782370,16.845841
sar2541-0004,-1.407345,416.075469,1371,1275,0.491459,14.094369,309.068806,17.580353


In [11]:
trends[trends.index.str.startswith("sar")].describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000
mean,-0.365266,224.936410,1371.0,1302.600000,0.113619,11.354878,225.883136,14.507046
std,0.604747,107.912166,0.0,26.538651,0.212013,4.119776,121.356899,4.391578
min,-1.407345,161.154798,1371.0,1275.000000,0.001673,6.446574,87.725117,9.366169
25%,-0.315984,165.718466,1371.0,1276.000000,0.007582,7.284243,102.875692,10.142766
50%,-0.207761,182.216593,1371.0,1305.000000,0.018165,14.094369,283.782370,16.845841
75%,0.031699,199.516726,1371.0,1328.000000,0.049214,14.115068,309.068806,17.580353
max,0.073058,416.075469,1371.0,1329.000000,0.491459,14.834138,345.963697,18.600099


In [12]:
trends.describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.000000
mean,-0.365266,224.936410,1371.0,1302.600000,0.113619,11.354878,225.883136,14.507046
std,0.604747,107.912166,0.0,26.538651,0.212013,4.119776,121.356899,4.391578
min,-1.407345,161.154798,1371.0,1275.000000,0.001673,6.446574,87.725117,9.366169
25%,-0.315984,165.718466,1371.0,1276.000000,0.007582,7.284243,102.875692,10.142766
50%,-0.207761,182.216593,1371.0,1305.000000,0.018165,14.094369,283.782370,16.845841
75%,0.031699,199.516726,1371.0,1328.000000,0.049214,14.115068,309.068806,17.580353
max,0.073058,416.075469,1371.0,1329.000000,0.491459,14.834138,345.963697,18.600099


In [13]:
(transects.trend - trends.trend).describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: trend, dtype: float64

In [14]:
trends["ERODIBILITY"]="High"

In [15]:
transects.update(trends.drop_duplicates())

In [16]:
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.315984,1371.0,1305.0,0.049214,14.115068,283.782370,16.845841,199.516726,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.031699,1371.0,1328.0,0.001673,6.446574,87.725117,9.366169,165.718466,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.073058,1371.0,1329.0,0.007582,7.284243,102.875692,10.142766,161.154798,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [17]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse', 'ERODIBILITY'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True]))

In [18]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.315984,1371.0,1305.0,0.049214,14.115068,283.782370,16.845841,199.516726,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.031699,1371.0,1328.0,0.001673,6.446574,87.725117,9.366169,165.718466,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.073058,1371.0,1329.0,0.007582,7.284243,102.875692,10.142766,161.154798,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [19]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.068364,664.0,508.0,0.000113,52.578021,6137.692249,78.343425,124.161754,Medium,"LINESTRING (8.40852 38.86175, 8.40882 38.86535)"
sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.197266,664.0,498.0,0.002798,19.969642,2010.059995,44.833693,197.183041,Medium,"LINESTRING (8.4084 38.86162, 8.41092 38.86464)"
sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.009191,664.0,545.0,0.000059,11.768691,215.362959,14.675250,206.410997,Medium,"LINESTRING (8.40893 38.86153, 8.41236 38.86393)"
sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.040081,664.0,597.0,0.002059,8.475977,112.442187,10.603876,239.306868,Medium,"LINESTRING (8.40904 38.86129, 8.41333 38.8626)"
sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.009671,664.0,603.0,0.000141,7.011139,96.998364,9.848775,258.388646,Medium,"LINESTRING (8.40904 38.86176, 8.4133 38.86037)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.315984,1371.0,1305.0,0.049214,14.115068,283.782370,16.845841,199.516726,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.031699,1371.0,1328.0,0.001673,6.446574,87.725117,9.366169,165.718466,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.073058,1371.0,1329.0,0.007582,7.284243,102.875692,10.142766,161.154798,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [20]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")